In [1]:
! pip install semantic-kernel

In [10]:
import semantic_kernel as sk
from dotenv import dotenv_values

config = dotenv_values()
kernel = sk.Kernel()

In [11]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel.add_service(
    AzureChatCompletion(
        service_id="az-chat-with-memory",
        deployment_name=config["AZURE_OPENAI_DEPLOYMENT"],
        api_key=config["AZURE_OPENAI_API_KEY"],
        api_version=config["AZURE_API_VERSION"],
        endpoint=config["AZURE_OPENAI_ENDPOINT"]
    )
)

In [12]:
import semantic_kernel.connectors.ai.open_ai as sk_oai
from semantic_kernel.prompt_template.input_variable import InputVariable

prompt = "ChatBot can have a conversation on any topic.\r\n{{$history}}\r\nUser: {{$userInput}}\r\nBot:"

execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
    service_id= "az-chat-with-memory",
    max_tokens= 200
)

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name= "chatWithHistory",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="userInput", description="The user input", is_required=True),
        InputVariable(name="history", description="Chat history", is_required=True),
    ],
    execution_settings=execution_settings
)

chatFunction = kernel.create_function_from_prompt(
    function_name="ChatWithHistory",
    plugin_name="chatWithHistoryPlugin",
    prompt_template_config=prompt_template_config
)

In [21]:
from semantic_kernel.functions.kernel_arguments import KernelArguments

history = ""
arguments = KernelArguments(history=history)

async def chatWithMe(userChatInput):
    global history

    arguments["userInput"] = userChatInput
    response = await kernel.invoke(chatFunction, arguments=arguments)
    result = f"\nUser: {userChatInput}\nChatBot: {response}\n"
    history += result
    arguments["history"] = history

    print(result)

In [22]:
while True:
    userInput = input("Input: ")
    await chatWithMe(userChatInput=userInput)

    if userInput == "exit":
        break


User: remember this pattern: 1 2 1 1 5 4 3
ChatBot: Got it! The pattern is: 1, 2, 1, 1, 5, 4, 3. How can I assist you with it?


User: whats the pattern
ChatBot: The pattern you provided is: 1, 2, 1, 1, 5, 4, 3. It appears to be a sequence of numbers, but without additional context, it's hard to determine a specific rule or relationship between them. If you have a specific question or if there's a particular way you'd like to analyze this pattern, let me know!


User: whats the pattern i sent
ChatBot: The pattern you sent is: 1, 2, 1, 1, 5, 4, 3. It seems to be a sequence of numbers, but without more context, it's difficult to identify a specific rule or meaning behind it. If you have a particular question or if there's a specific way you'd like to explore this pattern, feel free to share!


User: okay bet, sorry I was just checking to see if you are able to see the conversation history or not
ChatBot: No problem! I can remember the context of our conversation while we're chatting, bu

Hmmm, I thought the video would use some cloud services to contain the chat history, but it used a variable. So once the chat finishes then the chatBot cannot remember the past chats/conversations :(, will have to do my own studying I guess. But for now, I'll aim to finish the playlist of videos.